In [4]:
# 6. Plot volume
import numpy as np
from skimage.measure import block_reduce

pred1 = np.load("/Users/tommy/Projects/gap-junction-segmentation/data/volume_block_downsampled8x_5dggwboi.npy")
pred2 = np.load("/Users/tommy/Projects/gap-junction-segmentation/data/volume_block_downsampled8x_u4lqcs5g.npy")
pred1 = block_reduce(pred1, block_size=(1,2,2), func=np.max)
pred2 = block_reduce(pred2, block_size=(1,2,2), func=np.max)
pred1_ = np.transpose(pred1, (2, 1, 0))
pred2_ = np.transpose(pred2, (2, 1, 0))
#print(np.unique(pred_3d_, return_counts=True))

#### With Plotly (Currently Broken)

In [ ]:
import plotly.graph_objects as go
from skimage.measure import marching_cubes
import plotly.io as pio
from PIL import Image
import io
from scipy.spatial import Delaunay
import cv2

def simplify_mesh(vertices, faces, reduction_factor=0.5):
    """Reduce mesh complexity by decimation"""
    # Use PyVista's built-in decimation
    import pyvista as pv
    mesh = pv.PolyData(vertices, np.column_stack([np.full(len(faces), 3), faces]))
    decimated = mesh.decimate(reduction_factor)
    new_verts = decimated.points
    new_faces = decimated.faces.reshape(-1, 4)[:, 1:4]
    return new_verts, new_faces

# Function to create rotated frame
def create_frame(angle):
    # Rotate vertices around Z-axis
    cos_a = np.cos(np.radians(angle))
    sin_a = np.sin(np.radians(angle))
    
    # Center the meshes
    center1 = verts1.mean(axis=0)
    center2 = verts2.mean(axis=0)
    
    # Apply rotation
    v1_centered = verts1 - center1
    v1_rot = np.column_stack([
        v1_centered[:, 0] * cos_a - v1_centered[:, 1] * sin_a,
        v1_centered[:, 0] * sin_a + v1_centered[:, 1] * cos_a,
        v1_centered[:, 2]
    ]) + center1
    
    v2_centered = verts2 - center2
    v2_rot = np.column_stack([
        v2_centered[:, 0] * cos_a - v2_centered[:, 1] * sin_a,
        v2_centered[:, 0] * sin_a + v2_centered[:, 1] * cos_a,
        v2_centered[:, 2]
    ]) + center2
    
    # Offset second mesh for side-by-side
    offset = pred1_.shape[0] + 50
    
    fig = go.Figure()
    
    # Add first mesh
    fig.add_trace(go.Mesh3d(
        x=v1_rot[:, 0], y=v1_rot[:, 1], z=v1_rot[:, 2],
        i=faces1[:, 0], j=faces1[:, 1], k=faces1[:, 2],
        color='cyan',
        name='UNet 5dggwboi',
        opacity=0.8,
        lighting=dict(ambient=0.5, diffuse=0.8, specular=0.2)
    ))
    
    # Add second mesh
    fig.add_trace(go.Mesh3d(
        x=v2_rot[:, 0] + offset, y=v2_rot[:, 1], z=v2_rot[:, 2],
        i=faces2[:, 0], j=faces2[:, 1], k=faces2[:, 2],
        color='orange',
        name='UNet u4lqcs5g',
        opacity=0.8,
        lighting=dict(ambient=0.5, diffuse=0.8, specular=0.2)
    ))
    
    fig.update_layout(
        scene=dict(
            bgcolor='black',
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
            aspectmode='data'
        ),
        showlegend=True,
        width=1920,
        height=1080,
        paper_bgcolor='black',
        font=dict(color='white')
    )
    
    return fig

# Create mesh from volume
verts1, faces1, _, _ = marching_cubes(pred1_, level=127)
verts2, faces2, _, _ = marching_cubes(pred2_, level=127)

# Simplify before rendering
verts1, faces1 = simplify_mesh(verts1, faces1, reduction_factor=0.7)  # Keep 30% of triangles
verts2, faces2 = simplify_mesh(verts2, faces2, reduction_factor=0.7)
print(f"Simplified mesh sizes: {len(verts1)} and {len(verts2)} vertices")

# After generating frames as PIL Images
output_path = "/Users/tommy/Projects/gap-junction-segmentation/outputs/predictions_comparison.mp4"
height, width = 1080, 1920
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(output_path, fourcc, 30, (width, height))

print("Generating frames...")
for i in range(360):
    if i % 30 == 0:
        print(f"Frame {i}/360")
    
    angle = i
    fig = create_frame(angle)
    img_bytes = pio.to_image(fig, format='png', width=width, height=height)
    img = Image.open(io.BytesIO(img_bytes))
    frame = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    video.write(frame)

video.release()
print("Done!")

In [ ]:
# wrap into a pyvista UniformGrid
grid1 = pv.wrap(pred1_)
contour1 = grid1.contour(isosurfaces=[255])
grid2 = pv.wrap(pred2_)
contour2 = grid2.contour(isosurfaces=[255])

# # Volume rendering (interactive)
# pv.set_jupyter_backend("html")
# p = pv.Plotter(notebook=True)
# p.add_mesh(contour1, color="#02EBFC", show_scalar_bar=False)
# p.add_mesh(contour2, color="#BF35FF", show_scalar_bar=False)
# p.set_background("black")
# p.camera_position = "iso"
# p.enable_eye_dome_lighting()        
# p.show_axes()                         
# #p.export_html("/home/tommytang111/gap-junction-segmentation/html_objects/SEM_Dauer_1_3D.html")
# p.show()

#### Export interactive browser format with multiple subplots

In [ ]:
#Global plot settings
pv.set_jupyter_backend("client")
pv.global_theme.background = 'black'
pv.force_float=False
p = pv.Plotter(notebook=True, shape=(1, 2))

#Prediction 1
p.subplot(0,0)
p.add_mesh(contour1, color="#02EBFC", show_scalar_bar=False)
p.add_title("UNet 5dggwboi", color="white", font_size=26)
p.enable_eye_dome_lighting()
p.show_axes()
p.camera_position = "iso"
#Prediction 2
p.subplot(0, 1)
p.add_mesh(contour2, color="#FA9017", show_scalar_bar=False)
p.add_title("UNet u4lqcs5g", color="white", font_size=26)
p.enable_eye_dome_lighting()
p.show_axes()
p.camera_position = "iso"
p.link_views()
p.show()

#Check mesh stats before export
print(f"contour1: {contour1.n_points} points, {contour1.n_cells} cells")
print(f"contour2: {contour2.n_points} points, {contour2.n_cells} cells")
# print(f"spheres: {spheres.n_points} points, {spheres.n_cells} cells")
# print(f"spheres2: {spheres2.n_points} points, {spheres2.n_cells} cells")

# Export gltf
# gltf_path = "/home/tommytang111/gap-junction-segmentation/html_objects/scene2.gltf"
# p.export_gltf(gltf_path)

# # Create minimal HTML wrapper with model-viewer
# wrapper_path = "/home/tommytang111/gap-junction-segmentation/html_objects/scene.html"
# gltf_rel = "scene.gltf"
# html_wrapper = "\n".join([
# "<!DOCTYPE html><html><head><meta charset='utf-8'><title>Scene</title>",
# "<script type='module' src='https://ajax.googleapis.com/ajax/libs/model-viewer/3.3.0/model-viewer.min.js'></script>",
# "<style>",
# "  html,body{margin:0;height:100%;background:#000;}",
# "  model-viewer{width:100%;height:100%;}",
# "</style></head>",
# "<body>",
# f"<model-viewer src='{gltf_rel}' camera-controls touch-action='pan-y'",
# "  shadow-intensity='0' exposure='1.0' environment-image='neutral'",
# "  ar disable-tap",
# "  style='background-color:#000000'>",
# "</model-viewer>",
# "</body></html>"
# ])
# open(wrapper_path, "w").write(html_wrapper)
# print("Wrote:", wrapper_path)

Widget(value='<iframe src="http://localhost:50289/index.html?ui=P_0x12a75de10_1&reconnect=auto" class="pyvista…

contour1: 6014177 points, 12307314 cells
contour2: 6444998 points, 13072649 cells


In [ ]:
#Plotting only combined overlay
pv.set_jupyter_backend("html")
pv.global_theme.background = 'black'
pv.force_float=False

p = pv.Plotter(notebook=True)
p.add_mesh(contour, color="#02EBFC", opacity=0.5, show_scalar_bar=False)
p.add_mesh(contour2, color="#BF35FF", opacity=0.5, show_scalar_bar=False)
p.add_mesh(spheres, color="#FA9017", show_scalar_bar=False, smooth_shading=False)
p.add_text("Prediction + Point Entity + Points Overlay", position=(0, 0.5), color="#02EBFC", font_size=30)
p.enable_eye_dome_lighting()
p.show_axes()
#p.export_html("/home/tommytang111/gap-junction-segmentation/html_objects/SEM_Adult_point_entities_overlay_only.html")
p.show()

NameError: name 'contour' is not defined

In [ ]:
import sys
sys.path.append('/home/tommytang111/gap-junction-segmentation/code/src')
from utils import create_dataset_3d
create_dataset_3d(imgs_dir="/home/tommytang111/gap-junction-segmentation/data/sem_adult_larger/SEM_split/imgs", 
                  output_dir="/home/tommytang111/gap-junction-segmentation/data/972vols_sem_adult/vols")

Creating 3D volumes: 100%|██████████| 456/456 [00:04<00:00, 94.81it/s]


(1, 1, 456, (512, 512))

### Video Export Example

#### Load and transform

In [ ]:
import pyvista as pv
import numpy as np 
from skimage.measure import block_reduce
import os

#Load volumes
#pred = np.load("/home/tommytang111/gap-junction-segmentation/outputs/volumetric_results/unet_u4lqcs5g/sem_adult_s000-699/volume_block_downsampled4x.npy")
points = np.load("/Volumes/Samsung USB/sem_adult_CSs_block_downsampled8x.npy")
points2 = np.load("/Volumes/Samsung USB/sem_adult_moved_GJs_downsampled4x.npy")
#point_entities = np.load("/home/tommytang111/gap-junction-segmentation/gj_point_annotations/sem_adult_GJs_entities_downsampled8x.npy")

#Block reduce
# pred_ = block_reduce(pred, block_size=(1,4,4), func=np.max)
points_ = block_reduce(points, block_size=(1,1,1), func=np.max)
points2_ = block_reduce(points2, block_size=(1,2,2), func=np.max)
#point_entities_ = block_reduce(point_entities, block_size=(1,2,2), func=np.max)

#Tranpose to XYZ for pyvista
#pred_ = np.transpose(pred_, (2, 1, 0))
points_ = np.transpose(points_, (2, 1, 0))
points2_ = np.transpose(points2_, (2, 1, 0))

#### Convert to pyvista objects

In [ ]:
#Experimenting with overlaying objects
# print(pred_.shape)
print(points_.shape)
print(points2_.shape)
#print(point_entities_.shape)
#smaller_pred_3d = pred_3d[:200, 256:768, 256:768]
# print(np.unique(pred_3d, return_counts=True))
# print(np.unique(smaller_pred_3d, return_counts=True))

#Convert to list of points
points_list = np.argwhere(points_ == 255)
points_list2 = np.argwhere(points2_ == 255)

#Transform pred into isosurface
# grid = pv.wrap(pred_)
# contour = grid.contour(isosurfaces=[255])

#Transform point_entities into isosurface
# grid2 = pv.wrap(point_entities_)
# contour2 = grid2.contour(isosurfaces=[255])

#Transform points into glyphs/spheres
point_cloud = pv.PolyData(points_list)
point_cloud2 = pv.PolyData(points_list2)
lowpoly = pv.Sphere(radius=5.0, theta_resolution=8, phi_resolution=8)  # low triangle count
spheres = point_cloud.glyph(scale=False, geom=lowpoly, orient=False) #pv.Sphere(radius=2)
spheres2 = point_cloud2.glyph(scale=False, geom=lowpoly, orient=False) #pv.Sphere(radius=2)

# #Strip all unused data arrays before export and cast from float64 to float32 - reduces export size
# for m in (contour, contour2):
#     m.clear_data()  # remove point/cell data
#     # cast coordinates to float32
#     m.points = m.points.astype(np.float32, copy=False)

(2496, 1376, 700)
(2496, 1376, 700)


/Users/tommy/miniconda3/envs/gj/lib/python3.10/site-packages/pyvista/core/utilities/points.py:79: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1920, 1080) to (1920, 1088) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


#### Export video

In [ ]:
#Plotting only combined overlay
pv.set_jupyter_backend('client')
pv.global_theme.background = 'black'
pv.force_float=False

p = pv.Plotter(notebook=False, off_screen=True, window_size=[1920, 1080])
p.open_movie("~/Projects/gap-junction-segmentation/sem_adult_cs_and_gjs.mp4", framerate=30, quality=8)
# p.add_mesh(contour, color="#02EBFC", opacity=0.5, show_scalar_bar=False)
# p.add_mesh(contour2, color="#BF35FF", opacity=0.5, show_scalar_bar=False)
p.add_mesh(spheres, color="#FA9017", show_scalar_bar=False, smooth_shading=False)
p.add_mesh(spheres2, color="#02EBFC", show_scalar_bar=False, smooth_shading=False)
p.add_text("Chemical and Electrical Synapses", position=(0, 0), color="#FFFFFF", font_size=24, shadow=True, font="courier")
p.enable_eye_dome_lighting()
p.camera.zoom(1.2)
p.add_camera_orientation_widget()
#Scale bar
line = np.array([[1000, 650, 380], [1500, 850, 370]])
p.add_lines(line, color='w', width=25, connected=True, label="100 voxels")
#p.add_scalar_bar()
     
p.camera_position = [(3868.8407778921764, 292.5636480994484, 40.45188484086728),
                     (1349.5000610351562, 727.0, 344.0),
                     (-0.18233084170763672, -0.9763841917201768, -0.11588517731410687)]
p.show(auto_close=False)

#Write frames to video
n_frames = 720
for _ in range(n_frames): 
    p.camera.azimuth += 1
    #p.reset_camera_clipping_range()  # Keeps the flashing fix
    p.write_frame()
    
p.close()